In [1]:
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium_stealth import stealth
from pydantic import BaseModel
from selenium import webdriver
from typing import Optional

from redis_om import (
    Field,
    HashModel,
    Migrator,
    JsonModel
)

In [ ]:
def init_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('start-maximized')
    options.add_argument('--window-size=1920,1080')
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    stealth(driver,
        languages=["es-PE", "es"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
    )
    return driver

In [15]:
import redis

r = redis.Redis(
  host='redis-12622.c277.us-east-1-3.ec2.cloud.redislabs.com',
  port=12622,
  password='TxVYpEfg4DwZSjDxerOiWxNEhgIZouKa')

class Game(JsonModel):
    url: Optional[str] = Field(index=True, primary_key=True)
    site: Optional[str] = Field(index=True)
    sport: Optional[str] = Field(index=True)
    game: Optional[str] = Field(index=True, full_text_search=True, default='')
    firstTeam: Optional[str] = Field(index=True, full_text_search=True, default='')
    secoundTeam: Optional[str] = Field(index=True, full_text_search=True, default='')
    
    class Meta:
        database = r
        global_key_prefix = 'tracker'
        model_key_prefix = 'games'

Migrator().run()

In [ ]:
driver = init_driver()
driver.get('https://betway.com/es/sports/cat/soccer')

In [ ]:
for elm in driver.find_elements(By.CSS_SELECTOR, '.scoreboardInfoNames'):
    game = Game(
        url = elm.get_attribute('href'),
        site ='betway.com',
        sport = 'futbol',
        game = elm.text,
        firstTeam = elm.text.split('-')[0].strip(),
        secoundTeam = elm.text.split('-')[1].strip()
    )
    game.save()
    print(game.__dict__)

In [16]:

Migrator().run()
games = Game.find(Game.sport == 'futbol').all()

#for game in games:
#    Game.delete(game.url)
list(Game.all_pks())

[]

In [14]:

class Bid(BaseModel):
    channel: str = 'prueba'
    url: str = url
    group: str
    name: str
    bid: float

NameError: name 'url' is not defined

In [ ]:
import redis

r = redis.Redis(
    host='redis-12622.c277.us-east-1-3.ec2.cloud.redislabs.com',
    port=12622,
    password='TxVYpEfg4DwZSjDxerOiWxNEhgIZouKa'
)


In [ ]:
def get_bids():
    for elm in driver.find_elements(By.CSS_SELECTOR, '.collapsablePanel'):
        collapsed_elm = elm.find_element(By.CSS_SELECTOR, '.collapsableContent')
        if 'empty' in collapsed_elm.get_attribute('class'):
            try:
                elm.click()
            except:
                pass
        try:
            header_elm = elm.find_element(By.CSS_SELECTOR, '.titleText>.title')
            cashout_elm = elm.find_element(By.CSS_SELECTOR, '.cashOutMarketIndicatorContainer')
        except:
            continue    
        try:
            labels_elms = elm.find_elements(By.CSS_SELECTOR, '.outcomeHeader, .outcomeItemHeader')
            odds_elms = elm.find_elements(By.CSS_SELECTOR, '.oddsDisplay')
        except:
            continue
        
        for labels_elm, odds_elm in zip(labels_elms, odds_elms):
            try:
                bit = Bid(
                    group=header_elm.text, 
                    name=labels_elm.text, 
                    bid=float(odds_elm.text.replace(',', '.'))
                )
            except Exception as e:
                print('Error', e)
            else:
                print(bit)

In [ ]:
#get_bids()
driver.find_elements(By.CSS_SELECTOR, '.collapsablePanel')
driver.save_screenshot("image.png")

In [ ]:
import redis

r = redis.Redis(
  host='redis-12622.c277.us-east-1-3.ec2.cloud.redislabs.com',
  port=12622,
  password='TxVYpEfg4DwZSjDxerOiWxNEhgIZouKa')

In [ ]:
r.publish('betson', 'hello')